# Redes Neurais Auto Encoders

### Descriçao

---
1. O objetivo deste notebook é construir uma Rede Neural Auto Enconders.
2. Utilizaremos o conjunto de dados direto do github, a qual iremos baixar e usar.

3. O problema consiste em prever agrupar e criar sistemas de recomendações a partir de avaliações de filmes
---

### Dicionário do arquivo movies


Fields	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
indice_filme 	  										  	  |string     | Indice do filme |
nome_filme														  |string    | Título do Filme                        |
genero_filme		     										  |string     | Genero do filme               |

  

### Dicionário do arquivo user

Fields	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
indice_user 	  										  	  |string     | Indice do usuário |
genero_user														  |string    | Genero do usuário                      |
idade_user		     										  |int     | Idade do usuário               |
cod_profissao		     										  |int     | código da profissão               |
cep		     										  |int     | cep         |
  

### Dicionário do arquivo rating

Fields	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
ind_user 	  										  	  |int     | Coluna do usuário |
indice_filme														  |int    | Coluna do filme              |
avaliacao		     										  |int     | Avaliação (nota)              |
data		     										  |timestamp     | data em timestamp               |


# Instalação dos pacotes

In [ ]:
!pip install pandas numpy scikit-learn keras torch torchvision torchaudio



# Documentação

1. **Pandas** -> [Link](https://pandas.pydata.org/docs/)
2. **Numpy** -> [Link](https://numpy.org/doc/)
3. **Scikit Learn** -> [Link](https://scikit-learn.org/stable/)
4. **Keras** -> [Link](https://keras.io/api/)
5. **TensorFlow** -> [Link](https://www.tensorflow.org/api_docs/python/tf/keras)
6. **PyTorch** -> [Link](https://pytorch.org/docs/stable/index.html)


# Obtendo o dataset

In [ ]:
!git clone https://github.com/batestin1/coding_the_future_dio_redes_neurais.git #clona o repositorio
!mv coding_the_future_dio_redes_neurais/dataset /content/ #move apenas a pasta dataset para fora do diretorio
!rm -rf coding_the_future_dio_redes_neurais #exclui o restante que nao nos interessa



# Instalando as bibliotecas




In [ ]:
import numpy as np  # Biblioteca para operações numéricas e manipulação de arrays.
import pandas as pd  # Biblioteca para análise e manipulação de dados.
import os  # Biblioteca para interações com o sistema operacional, como manipulação de arquivos e diretórios.
import torch  # Biblioteca principal do PyTorch para operações de tensor.
import torch.nn as nn  # Submódulo do PyTorch para a construção de redes neurais.
import torch.nn.parallel  # Submódulo do PyTorch para treinamento paralelo de redes neurais.
import torch.optim as optim  # Submódulo do PyTorch para algoritmos de otimização.
import torch.utils.data  # Submódulo do PyTorch para manipulação de conjuntos de dados.
from torch.autograd import Variable  # Classe do PyTorch para autograd, que permite a diferenciação automática de operações em tensores.
from keras.models import load_model  # Função para carregar modelos pré-treinados do Keras.
from tensorflow.keras.models import load_model #para salvar modelos do keras




# Lendo os Dataset

In [ ]:
#Importando os dados
movies = pd.read_csv('/content/dataset/ae/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/content/dataset/ae/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/content/dataset/ae/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [ ]:
movies.head(3)

In [ ]:
movies.columns = ['indice_filme', 'nome_filme', 'genero_filme']
movies.head(3)

In [ ]:
users.head()

In [ ]:
users.columns = ['indice_user', 'genero_user', 'idade_user', 'cod_profissao', 'cep']
users.head()

In [ ]:
ratings.head()

In [ ]:
ratings.columns = ['ind_user', 'indice_filme', 'avaliacao', 'data']
ratings.head()

In [ ]:
visualizando_data = ratings.copy()
visualizando_data['data'] = pd.to_datetime(visualizando_data['data'], unit='s')
visualizando_data.head()

# Preparando os dados de treino e os dados de teste

In [ ]:
training_set = pd.read_csv('/content/dataset/ae/train.csv') # Carrega o conjunto de dados de treinamento a partir de um arquivo CSV
training_set = np.array(training_set, dtype = 'int') # Converte o conjunto de dados de treinamento em um array NumPy com o tipo de dados 'int'
test_set = pd.read_csv('/content/dataset/ae/test.csv') # Carrega o conjunto de dados de teste a partir de um arquivo CSV
test_set = np.array(test_set, dtype = 'int') # Converte o conjunto de dados de teste em um array NumPy com o tipo de dados 'int'

In [ ]:
training_set.shape

In [ ]:
training_set[0]

In [ ]:
test_set.shape

In [ ]:
test_set[0]

In [ ]:
training_set[:,0].max()

In [ ]:
test_set[:,1].max()

In [ ]:
#Quantidade de usuários e filmes
nb_users = int(training_set[:,0].max()) # Encontra o maior ID de usuário no conjunto de treinamento
nb_movies = int(training_set[:,1].max()) # Encontra o maior ID de filme  no conjunto de treinamento

In [ ]:
nb_users

In [ ]:
nb_movies

In [ ]:
# Função para converter os dados em uma matriz com usuários nas linhas e filmes nas colunas\
#aqui vamos fazer uma lista para cada usuario que vai ter 6040 linhas e 3952 colunas
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):  # Itera sobre cada ID de usuário
        id_movies = data[:,1][data[:,0] == id_users]  # Obtém os IDs dos filmes avaliados pelo usuário atual
        id_ratings = data[:,2][data[:,0] == id_users]  # Obtém as avaliações dos filmes feitas pelo usuário atual
        ratings = np.zeros(nb_movies)  # Cria um vetor de zeros com o tamanho do número total de filmes
        ratings[id_movies - 1] = id_ratings  # Atribui as avaliações aos índices correspondentes dos filmes
        new_data.append(list(ratings))  # Adiciona o vetor de avaliações à nova lista de dados
    return new_data  # Retorna a nova matriz de dados

# Converte o conjunto de dados de treinamento usando a função definida
training_set = convert(training_set)

# Converte o conjunto de dados de teste usando a função definida
test_set = convert(test_set)


In [ ]:
type(training_set)

In [ ]:
len(training_set)

In [ ]:
len(test_set)

#### Essa padronizacao do mesmo tamanho do arquivo de treino e teste significa que apesar do usuario ter ou não avaliado o filme, ele vai estar no arquivo e nossa rede vai aprender com isso. caso nao tenha avaliado, preenchemos com 0

# Criando Tensors do Torch


##### Aqui vamos criar os tensores do toach. Matrizes multidimensionais de um unico tipo, em nosso caso, float

In [ ]:
training_set = torch.FloatTensor(training_set) # Converte o conjunto de dados de treinamento para um tensor de ponto flutuante do PyTorch
test_set = torch.FloatTensor(test_set) # Converte o conjunto de dados de teste para um tensor de ponto flutuante do PyTorch

In [ ]:
training_set

# Criando a arquitetura da AE

In [ ]:


class redes_auto_enconder(nn.Module): # Vamos utilizar o conceito de herança para empilhar (stack) nossa rede de autoencoder
    def __init__(self, ):
        super(redes_auto_enconder, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20) # Define a primeira camada totalmente valor total de filmes que definimos no 'nb_movies', e o 20 é o numero de caracteristicas que vc define para o autoencoders detectar.
        self.fc2 = nn.Linear(20, 10) # Define a segunda camada totalmente conectada com 20 entradas (que sao os valores da saida anterior) e 10 saídas
        self.fc3 = nn.Linear(10, 20) # Define a terceira camada totalmente conectada com 10 entradas e 20 saídas
        self.fc4 = nn.Linear(20, nb_movies) # Define a quarta camada totalmente conectada com 20 entradas e 'nb_movies' saídas
        self.activation = nn.Sigmoid()  # Define a função de ativação Sigmoid que será usada após cada camada totalmente conectada

    def forward(self, x):

        x = self.activation(self.fc1(x)) # X é o vector de entrada. Ela ativia os neuronios da primeira camada. Aplica a primeira camada e a função de ativação Sigmoid
        x = self.activation(self.fc2(x))  # Aplica a segunda camada e a função de ativação Sigmoid
        x = self.activation(self.fc3(x))  # Aplica a terceira camada e a função de ativação Sigmoid
        x = self.fc4(x) # Aplica a quarta camada (sem função de ativação)
        return x #retorna o valor do vector


rae = redes_auto_enconder() # Instancia a rede autoencoder


criterion = nn.MSELoss() # metodo que será usado para metrificar o error. a metrica é MSE

# Este otimizador vai ativar o Stochastic Gradient Descent para atualizar os pesos e reduzir o erro.
optimizer = optim.RMSprop(rae.parameters(), # estamos utilizando o metodo parameters do torch que traz uma serie automatica de parametros do objeto.
                          lr=0.01,          # é a taxa de aprendizado
                          weight_decay=0.5) # Aqui é taxa de decaimento de peso


# Treinando a AE

In [ ]:

nb_epoch = 8  # Define o número de épocas (iterações completas sobre o conjunto de dados) como 8, mas recomendo que em casa você use 800

for epoch in range(1, nb_epoch + 1):  # Loop para cada época

    train_loss = 0  # Variavel que armazena a taxa de perda, iniciando em 0
    s = 0.0  # Vai contar o numero de usuario que avaliaram pelo menos 1 filme.

    for id_user in range(nb_users):  # Loop sobre cada usuário

        input = Variable(training_set[id_user]).unsqueeze(0)  # Cria uma variável a partir do conjunto de treinamento do usuário atual e adiciona uma dimensão, pq o pytorch nao aceita variavel de uma unica dimensao.

        target = input.clone() #target é o valor real, e por isso recebe nossa entrada (input) clonada.

        # este if só vai analisar usuarios que avaliaram o filme. Por isso o target.data é maior que 0 para pegar estes usuarios
        if torch.sum(target.data > 0) > 0:  # Verifica se há alguma avaliação (dado > 0) para o usuário atual
            output = rae(input)  # Passa o input através da rede autoencoder para obter o output
            target.require_grad = False  # Desabilita o cálculo do gradiente para o target
            output[target == 0] = 0  # Define as saídas correspondentes a zero no target também como zero no output
            loss = criterion(output, target)  # Calcula a perda entre o output e o target usando a função de perda definida
            # Calcula um fator de correção para a média, dividindo o número total de filmes pelo número de filmes avaliados pelo usuário
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()  # Realiza a retropropagação do erro para ajustar os pesos da rede
            # Acumula a perda de treino corrigida pela média dos filmes avaliados
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.  # Incrementa o contador de usuários com dados válidos
            optimizer.step()  # Atualiza os pesos da rede de acordo com o otimizador
    # Imprime o número da época e a perda média de treino para a época atual
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))


# Testando a Rede

In [ ]:
test_loss = 0  # Inicializa a variável de perda de teste
s = 0.  # Inicializa a variável de contagem de usuários com dados válidos para o teste
for id_user in range(nb_users):  # Loop sobre cada usuário
    # Cria uma variável a partir do conjunto de treinamento do usuário atual e adiciona uma dimensão
    input = Variable(training_set[id_user]).unsqueeze(0)
    # Cria uma variável a partir do conjunto de teste do usuário atual e adiciona uma dimensão
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:  # Verifica se há alguma avaliação (dado > 0) para o usuário atual no conjunto de teste
        output = rae(input)  # Passa o input através da rede autoencoder para obter o output
        target.require_grad = False  # Desabilita o cálculo do gradiente para o target
        output[target == 0] = 0  # Define as saídas correspondentes a zero no target também como zero no output
        loss = criterion(output, target)  # Calcula a perda entre o output e o target usando a função de perda definida
        mean_corrector = nb_movies / float(torch.sum(target.data > 0)) # Calcula um fator de correção para a média, dividindo o número total de filmes pelo número de filmes avaliados pelo usuário
        loss.backward() #esse parametro vai dizer em que direcao os pesos vao ser atualizados (se diminui ou aumenta)
        test_loss += np.sqrt(loss.data * mean_corrector) # Acumula a perda de teste corrigida pela média dos filmes avaliados
        s += 1.  # Incrementa o contador de usuários com dados válidos

# Imprime a perda média de teste para todos os usuários
print('Loss de teste: ' + str(test_loss / s))


# Salvando o modelo

In [ ]:
folder = 'ae/'

# Verifica se o diretório existe e, se não existir, cria o diretório
if not os.path.exists(folder):
    os.makedirs(folder)


# Salva o modelo no diretório especificado
torch.save(rae.state_dict(), os.path.join(folder, 'modelo_autoencoder.pth'))

# Importando os modelos

In [ ]:
# Carrega o estado do modelo a partir do arquivo salvo no diretório especificado
# Antes de carregar o modelo, você precisa instanciar uma nova rede_autoencoder
model = redes_auto_enconder()
model.eval()
# Carrega os pesos salvos do modelo
model.load_state_dict(torch.load(os.path.join(folder, 'modelo_autoencoder.pth')))


In [ ]:
model

In [ ]:
test_loss = 0
s = 0.0

for id_user in range(nb_users):
    input = Variable(test_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)

    if torch.sum(target.data > 0) > 0:
        output = rae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data * mean_corrector)
        s += 1.0

print('Test loss: ' + str(test_loss / s))